## Computing Cohen's Kappa Scores for Interrater Agreement

In [2]:
%pip install tabulate
%pip install seaborn
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:

import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from scipy.stats import kendalltau

# Read the data from the TSV files
df_rater1 = pd.read_csv('./data/bugs_analysis_rater_1.tsv', sep='\t').replace(np.nan, '', regex=True)
df_rater2 = pd.read_csv('./data/bugs_analysis_rater_2.tsv', sep='\t').replace(np.nan, '', regex=True)

print(len(df_rater1))
df_rater1.head()

60


,Rater,Skipped?,False pos?,Owner,Project,Title,Link,Fix Link,Stars,TF-IDFs,...,Erroneous Computation 1,Erroneous Computation 2,Erroneous Computation 3,Obscurity,Impact,# LOC,Logic Needed,Bug Pattern 1,Bug Pattern 2,Bug Pattern 3
0,5: Peter Sasha,False,False,frictionlessdata,frictionless-py,SpssParser ignores timezones,https://github.com/frictionlessdata/frictionle...,https://github.com/frictionlessdata/frictionle...,683.0,1.270077,...,Datetime Construction,String Parsing/Formatting (data),,Medium,Low,Medium,Medium,Dropping Timezones,,
1,2: Shrey Serena,False,False,googleapis,python-storage,`DeprecationWarning` warning in build log for ...,https://github.com/googleapis/python-storage/i...,https://github.com/googleapis/python-storage/c...,420.0,1.176351,...,Datetime Construction,,,Low,Low,Low,Low,Outdated/Deprecated APIs,,
2,3: Shrey Sasha,False,True,sdispater,pendulum,DateTime.add(duration),https://github.com/sdispater/pendulum/issues/323,https://github.com/gridsingularity/gsy-e/pull/...,6109.0,0.996023,...,,,,,,,,,,
3,2: Shrey Serena,False,False,Bears-R-Us,arkouda,datetime columns in dataframe display bug,https://github.com/Bears-R-Us/arkouda/issues/2596,https://github.com/stress-tess/arkouda/commit/...,228.0,0.947325,...,Datetime Construction,,,Medium,Low,Low,Low,Incorrect API usage,,
4,5: Peter Sasha,False,False,dateutil,dateutil,last day of the month is wrong??,https://github.com/dateutil/dateutil/issues/1167,https://github.com/dateutil/dateutil/pull/1168,2266.0,0.935691,...,Datetime Arithmetic,,,Low,Low,Medium,Medium,Unintuitive Arithmetic,,


In [14]:

# Helper function to compute the Cohen's Kappa score
def compute_cohen_kappa_score(df_rater1, df_rater2, col):
    reviewer_pairs = [
        "1: Shrey Peter",
        "2: Shrey Serena",
        "3: Shrey Sasha",
        "4: Peter Serena",
        "5: Peter Sasha",
        "6: Serena Sasha"
    ]

    scores = []
    for reviewer_pair in reviewer_pairs:
        df1 = df_rater1[df_rater1['Rater'] == reviewer_pair]
        df2 = df_rater2[df_rater2['Rater'] == reviewer_pair]

        df1_new = df1[(df1['False pos?'] == df2['False pos?']) & (df1['False pos?'] == False)]
        df2_new = df2[(df1['False pos?'] == df2['False pos?']) & (df2['False pos?'] == False)]

        print(f"Reviewer Pair: {reviewer_pair}")
        print(f"Cohen's Kappa Score: {cohen_kappa_score(df1_new[col], df2_new[col])}")
        print()

        scores.append(cohen_kappa_score(df1_new[col], df2_new[col]))

    print(f"Mean Cohen's Kappa Score: {np.mean(scores)}")
    # return np.mean(scores)

# Helper function to compute the weighted Cohen's Kappa score
def compute_weighted_cohen_kappa_score(df_rater1, df_rater2, col):
    reviewer_pairs = [
        "1: Shrey Peter",
        "2: Shrey Serena",
        "3: Shrey Sasha",
        "4: Peter Serena",
        "5: Peter Sasha",
        "6: Serena Sasha"
    ]

    scores = []
    for reviewer_pair in reviewer_pairs:
        df1 = df_rater1[df_rater1['Rater'] == reviewer_pair]
        df2 = df_rater2[df_rater2['Rater'] == reviewer_pair]

        df1_new = df1[(df1['False pos?'] == df2['False pos?']) & (df1['False pos?'] == False)]
        df2_new = df2[(df1['False pos?'] == df2['False pos?']) & (df2['False pos?'] == False)]

        print(f"Reviewer Pair: {reviewer_pair}")
        print(f"Cohen's Kappa Score: {cohen_kappa_score(df1_new[col], df2_new[col], weights='quadratic')}")
        print()

        scores.append(cohen_kappa_score(df1_new[col], df2_new[col], weights='quadratic'))

    print(f"Mean Cohen's Kappa Score: {np.mean(scores)}")
    # return np.mean(scores)

In [15]:
# Compute the Cohen's Kappa score for Category 1
compute_cohen_kappa_score(df_rater1, df_rater2, 'Category 1')

Reviewer Pair: 1: Shrey Peter
Cohen's Kappa Score: 0.5454545454545454

Reviewer Pair: 2: Shrey Serena
Cohen's Kappa Score: 1.0

Reviewer Pair: 3: Shrey Sasha
Cohen's Kappa Score: 0.0

Reviewer Pair: 4: Peter Serena
Cohen's Kappa Score: 1.0

Reviewer Pair: 5: Peter Sasha
Cohen's Kappa Score: 0.7586206896551724

Reviewer Pair: 6: Serena Sasha
Cohen's Kappa Score: 0.6

Mean Cohen's Kappa Score: 0.6506792058516196


In [16]:
# Compute the Cohen's Kappa score for Erroneous Computation 1
compute_cohen_kappa_score(df_rater1, df_rater2, 'Erroneous Computation 1')

Reviewer Pair: 1: Shrey Peter
Cohen's Kappa Score: 0.6666666666666667

Reviewer Pair: 2: Shrey Serena
Cohen's Kappa Score: 1.0

Reviewer Pair: 3: Shrey Sasha
Cohen's Kappa Score: 0.5555555555555556

Reviewer Pair: 4: Peter Serena
Cohen's Kappa Score: 0.84

Reviewer Pair: 5: Peter Sasha
Cohen's Kappa Score: 0.48780487804878037

Reviewer Pair: 6: Serena Sasha
Cohen's Kappa Score: 0.5199999999999999

Mean Cohen's Kappa Score: 0.6783378500451671


In [17]:
# Compute the Cohen's Kappa score for Obscurity
# Convert ordinal data to numerical data
mapping = {'Low': 1, 'Medium': 2, 'High': 3}

df_rater1["mapped_obscurity"] = df_rater1['Obscurity'].map(mapping)
df_rater2["mapped_obscurity"] = df_rater2['Obscurity'].map(mapping)

compute_weighted_cohen_kappa_score(df_rater1, df_rater2, 'mapped_obscurity')

Reviewer Pair: 1: Shrey Peter
Cohen's Kappa Score: 0.44444444444444453

Reviewer Pair: 2: Shrey Serena
Cohen's Kappa Score: 0.11111111111111116

Reviewer Pair: 3: Shrey Sasha
Cohen's Kappa Score: 0.6363636363636364

Reviewer Pair: 4: Peter Serena
Cohen's Kappa Score: 1.0

Reviewer Pair: 5: Peter Sasha
Cohen's Kappa Score: 0.41666666666666663

Reviewer Pair: 6: Serena Sasha
Cohen's Kappa Score: 0.33333333333333337

Mean Cohen's Kappa Score: 0.4903198653198653


In [19]:
# Compute the Cohen's Kappa score for Impact/Severity
# Convert ordinal data to numerical data
mapping = {'Low': 1, 'Medium': 2, 'High': 3}

df_rater1["mapped_impact_severity"] = df_rater1['Impact'].map(mapping)
df_rater2["mapped_impact_severity"] = df_rater2['Impact'].map(mapping)

compute_weighted_cohen_kappa_score(df_rater1, df_rater2, 'mapped_impact_severity')

Reviewer Pair: 1: Shrey Peter
Cohen's Kappa Score: 0.5161290322580645

Reviewer Pair: 2: Shrey Serena
Cohen's Kappa Score: 0.75

Reviewer Pair: 3: Shrey Sasha
Cohen's Kappa Score: 0.6666666666666667

Reviewer Pair: 4: Peter Serena
Cohen's Kappa Score: 1.0

Reviewer Pair: 5: Peter Sasha
Cohen's Kappa Score: -0.23529411764705888

Reviewer Pair: 6: Serena Sasha
Cohen's Kappa Score: 0.6666666666666667

Mean Cohen's Kappa Score: 0.5606947079907232


In [20]:
# Compute the Cohen's Kappa score for # LOC
# Convert ordinal data to numerical data
mapping = {'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4}

df_rater1["mapped_loc"] = df_rater1['# LOC'].map(mapping)
df_rater2["mapped_loc"] = df_rater2['# LOC'].map(mapping)

compute_weighted_cohen_kappa_score(df_rater1, df_rater2, 'mapped_loc')

Reviewer Pair: 1: Shrey Peter
Cohen's Kappa Score: 0.6153846153846154

Reviewer Pair: 2: Shrey Serena
Cohen's Kappa Score: 0.0

Reviewer Pair: 3: Shrey Sasha
Cohen's Kappa Score: 1.0

Reviewer Pair: 4: Peter Serena
Cohen's Kappa Score: 0.8367346938775511

Reviewer Pair: 5: Peter Sasha
Cohen's Kappa Score: 0.7341772151898733

Reviewer Pair: 6: Serena Sasha
Cohen's Kappa Score: 0.7931034482758621

Mean Cohen's Kappa Score: 0.6632333287879837


In [21]:
# Compute the Cohen's Kappa score for Logic Needed
# Convert ordinal data to numerical data
mapping = {'Low': 1, 'Medium': 2, 'High': 3}

df_rater1["mapped_logic_needed"] = df_rater1['Logic Needed'].map(mapping)
df_rater2["mapped_logic_needed"] = df_rater2['Logic Needed'].map(mapping)

compute_weighted_cohen_kappa_score(df_rater1, df_rater2, 'mapped_logic_needed')

Reviewer Pair: 1: Shrey Peter
Cohen's Kappa Score: 0.5454545454545454

Reviewer Pair: 2: Shrey Serena
Cohen's Kappa Score: 1.0

Reviewer Pair: 3: Shrey Sasha
Cohen's Kappa Score: -0.33333333333333326

Reviewer Pair: 4: Peter Serena
Cohen's Kappa Score: 0.6

Reviewer Pair: 5: Peter Sasha
Cohen's Kappa Score: 0.8

Reviewer Pair: 6: Serena Sasha
Cohen's Kappa Score: 0.5714285714285714

Mean Cohen's Kappa Score: 0.5305916305916306


In [23]:
# Compute the Cohen's Kappa score for BPST1
compute_cohen_kappa_score(df_rater1, df_rater2, 'Bug Pattern 1')

Reviewer Pair: 1: Shrey Peter
Cohen's Kappa Score: 0.4444444444444444

Reviewer Pair: 2: Shrey Serena
Cohen's Kappa Score: 0.6

Reviewer Pair: 3: Shrey Sasha
Cohen's Kappa Score: 0.6923076923076923

Reviewer Pair: 4: Peter Serena
Cohen's Kappa Score: 1.0

Reviewer Pair: 5: Peter Sasha
Cohen's Kappa Score: 0.6585365853658536

Reviewer Pair: 6: Serena Sasha
Cohen's Kappa Score: 0.0

Mean Cohen's Kappa Score: 0.5658814536863317
